In [1]:
import warnings
warnings.filterwarnings("ignore")

import functools, operator, requests, os, json
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
from IPython.display import Markdown

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain_experimental.tools import PythonREPLTool

USER_AGENT environment variable not set, consider setting it to identify your requests.


#### Load environment variables

In [2]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")

EMBEDDING_DEPLOYMENT_NAME = os.getenv('EMBEDDING_DEPLOYMENT_NAME')
CHATGPT_16K_DEPLOYMENT_NAME = os.getenv('CHATGPT_16K_DEPLOYMENT_NAME')
CHATGPT_TURBO_DEPLOYMENT_NAME = os.getenv('CHATGPT_TURBO_DEPLOYMENT_NAME')
GPT4_DEPLOYMENT_NAME = os.getenv('GPT4_DEPLOYMENT_NAME')
GPT4_TURBO_DEPLOYMENT_NAME = os.getenv('GPT4_TURBO_DEPLOYMENT_NAME')

API_KEY = os.getenv('southindia_API_KEY')
API_BASE = os.getenv('southindia_API_BASE')
API_TYPE = os.getenv('API_TYPE')
API_VERSION = os.getenv('API_VERSION')

#### Define LLM

#### OpenAI Style

In [3]:
# from langchain.chat_models import ChatOpenAI
# from langchain.embeddings.openai import OpenAIEmbeddings

# llm  = ChatOpenAI()
# embeddings = OpenAIEmbeddings()

#### Azure Style

In [15]:
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDING_DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    azure_endpoint=API_BASE,
    openai_api_version=API_VERSION,
)

llm = AzureChatOpenAI(
        azure_endpoint=API_BASE,
        openai_api_version=API_VERSION,
        azure_deployment=GPT4_TURBO_DEPLOYMENT_NAME,
        openai_api_key=API_KEY,
        openai_api_type=API_TYPE,
        temperature=0)

In [16]:
llm.invoke("What is 2+2 ?")

AIMessage(content='2 + 2 equals 4.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_811936bd4f', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-c19dd54b-6009-4fbd-96a7-4a9debddc152-0')

#### Agent Creation

In [6]:
import os
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain.agents import load_tools


# Define tools
def get_tavily_search_retriever():
    retriever = TavilySearchAPIRetriever(k=7)
    retriever_description = "Use this tool when you need to answer questions related to current events and latest happenings or anything related to the user ask"
    retrieve_tool = create_retriever_tool(retriever=retriever, name="web_search", description=retriever_description)
    return retrieve_tool

tavily_search_tool = get_tavily_search_retriever()
tools = load_tools(["llm-math"], llm=llm)
tools = tools + [tavily_search_tool]

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize the agent with verbose output
agent = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

/home/nalabsr1/venv/gen.ai_venv_39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [13]:
agent.run(input="Which is hotter today? Hyderabad or Delhi?")



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: google_search
Action Input: current temperature in Hyderabad, current temperature in Delhi
```
Observation: Current weather in Hyderabad, Telangana, India. Check current conditions in Hyderabad, Telangana, India with radar, hourly, and more.

Get the latest weather forecast for Hyderabad, including warnings, temperature, rain, wind and visibility. 14-day weather forecast for Hyderabad.

Today's Weather Report. Date: 2024-07-20. Station. Max Temp ( o C) Dep. from Normal. Min Temp ( o C) Dep. from Normal. RH at 0830IST. RH at 1730IST.

Current weather in Hyderabad and forecast for today, tomorrow, and next 14 days

Current Hyderabad weather forecast is Light rain with real-time temperature (25°C), humidity (84%), wind (16km/h), pressure (1003mb), UV (1%), visibility (5km) in Telangana. AQI; Ranking. ... 706, 7th Floor, Crown Heights, Rohini Sec-10, Delhi 110085, INDIA; GET OUR APP. FOLLOW US

Hydera

'Based on the information obtained, Delhi is currently hotter with a temperature of 30°C (86°F) compared to Hyderabad, where the current temperature is 25°C (77°F).'